In [1]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask
import intake
import collections
import fsspec
import seaborn as sns
from xmip.preprocessing import combined_preprocessing

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

In [2]:
def match_lat_lon_names(ds):
    for lat_name in ['y', 'latitude', 'nav_lat']:
        if (lat_name in ds.coords) and ('lat' not in ds.coords):
            ds=ds.rename({lat_name: 'lat'})
        else:
            ds=ds
    for lon_name in ['x', 'longitude', 'nav_lon']:
        if (lon_name in ds.coords) and ('lon' not in ds.coords):
            ds = ds.rename({lon_name: 'lon'})
        else:
            ds=ds
    return ds

def add_source_id_coord(ds):
    ds = ds.assign_coords(source_id=ds.attrs.get('source_id'))
    return ds

def annual_climatology(ds):
    # First calculate the monthly climatologies
    ds_avg = ds.groupby('time.month').mean(dim='time', keep_attrs=True).mean(dim='month', keep_attrs=True)
    return(ds_avg)

def monthly_climatology(ds):
    # First calculate the monthly climatologies
    ds_mon = ds.groupby('time.month').mean(dim='time', keep_attrs=True)
    return(ds_mon)

In [3]:
## Open up the PANGEO-CMIP6 repository

url='https://storage.googleapis.com/cmip6/pangeo-cmip6.json'
cmip6=intake.open_esm_datastore(url)

Not sure if I should be using historical or piControl experiments. Limiting the analysis to just the first realization (r1) from each group significantly reduces the amount of data to pull.

### Precip

In [10]:
## Search the repository

# Define query info
query = dict(activity_id='CMIP',
             experiment_id='historical',
             table_id='Amon',
             variable_id='pr',
             member_id = 'r1i1p1f1' )

# extract info for subset of models that match query
subset = cmip6.search(require_all_on=["source_id"], **query)

# print verbose list of results
subset.df 
# print compact list of results
#subset.df.groupby("source_id")[["experiment_id", "variable_id", "table_id"]].nunique()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,CSIRO-ARCCSS,ACCESS-CM2,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/...,NaN,20191108
1,CMIP,CSIRO,ACCESS-ESM1-5,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/hist...,NaN,20191115
2,CMIP,AWI,AWI-CM-1-1-MR,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/histor...,NaN,20200511
3,CMIP,AWI,AWI-ESM-1-1-LR,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/histo...,NaN,20200212
4,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/historic...,NaN,20181126
5,CMIP,BCC,BCC-ESM1,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/BCC/BCC-ESM1/historical/...,NaN,20181214
6,CMIP,CAMS,CAMS-CSM1-0,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/CAMS/CAMS-CSM1-0/histori...,NaN,20190708
7,CMIP,CAS,CAS-ESM2-0,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/CAS/CAS-ESM2-0/historica...,NaN,20201227
8,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190401
9,CMIP,NCAR,CESM2-FV2,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2-FV2/historica...,NaN,20191120


In [16]:
## Store results in a dataset dictionary
zarr_kwargs={'consolidated': True,
             'decode_times': False}
subset_dict = subset.to_dataset_dict(**zarr_kwargs) 
#list(subset_dict.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [227]:
## Create a new dictionary for manipulating

# initialize
cmip6_pr=dict()
models_to_del=[]
nlat_threshold=90

for key in subset_dict.keys():
    ## create shortened key name that only includes institution and source id
    _, inst_id, src_id, _, _, _ = key.split(sep='.')
    short_key=inst_id + '.' + src_id
    
    ## clean up dataset info
    ds=subset_dict[key].pr[0] # specifying 0 uses only the first member_id
    # make sure names of coordinate variables match
    ds=match_lat_lon_names(ds)
    # make a list of models with low resolution (<2.5°), for later
    if np.size(ds.lat)<nlat_threshold:
        models_to_del.append(short_key)
    # add a source_id coordinate
    ds=add_source_id_coord(subset_dict[key]) #ds.assign_coords(source_id=src_id)
    
    ## process data
    # convert precip to mm/day
    ds=ds*86400
    # Trim the time range
    #ds=ds.sel(time=slice('1948','2014'))
    # calculate time-mean
    ds=monthly_climatology(ds)
    
    ## update new dictionary with processed data
    cmip6_pr[short_key]=ds

# delete low res models
for model in models_to_del:
    del cmip6_pr[model]
# also delete this one because its not on a regular grid?
del cmip6_pr['MPI-M.ICON-ESM-LR'] 

In [213]:
## Create a new dictionary for storing regridded data

# initialize
cmip6_pr_same_grid=dict()

# Define standard coordinates based on lowest resolution model
lats = cmip6_pr['NASA-GISS.GISS-E2-1-G'].lat.values
lons = cmip6_pr['NASA-GISS.GISS-E2-1-G'].lon.values

# interp model grids to lower resolution
# all models are on a 0:360 grid so no need to convert lons
for key,ds in cmip6_pr.items():
    ds_lr=ds.interp(lat=lats, lon=lons)
    ds_lr_clean=drop_all_bounds(ds_lr)
    cmip6_pr_same_grid[key]=ds_lr_clean
    #cmip6_pr_same_grid=xr.concat([cmip6_pr_same_grid,ds_lr], dim='source_id', coords='minimal')

In [237]:
## Create a new xarray dataset for storing regridded data

# initialize
cmip6_pr_same_grid=[]

for index, (key, ds) in enumerate(cmip6_pr.items()):
    # interp to standard grid
    ds_lr=ds.interp(lat=lats, lon=lons)
    ds_lr_clean=drop_all_bounds(ds_lr)
    # concatenate
    if index==0:
        cmip6_pr_same_grid=ds_lr_clean
    else:
        cmip6_pr_same_grid=xr.concat([cmip6_pr_same_grid,ds_lr_clean], dim='source_id', coords='minimal')

# clean
cmip6_pr_same_grid=cmip6_pr_same_grid.drop(['bnds','member_id','dcpp_init_year'])
cmip6_pr_same_grid=cmip6_pr_same_grid.drop_dims(['bnds','member_id','dcpp_init_year'])

In [ ]:
# save output
cmip6_pr_same_grid.to_netcdf('cmip6.pr.climo.nc')

### SST

In [4]:
## Search the repository

# Define query info
query = dict(activity_id='CMIP',
             experiment_id='historical',
             table_id='Omon',
             variable_id='tos',
             member_id = 'r1i1p1f1' )

# extract info for subset of models that match query
subset = cmip6.search(require_all_on=["source_id"], **query)

# print verbose list of results
#subset.df 
# print compact list of results
#subset.df.groupby("source_id")[["experiment_id", "variable_id", "table_id"]].nunique()

In [5]:
## Store results in a dataset dictionary
zarr_kwargs={'consolidated': True,
             'decode_times': False}
subset_dict = subset.to_dataset_dict(**zarr_kwargs) 
#list(subset_dict.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


ESMDataSourceError: Failed to load dataset with key='CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn'
                 You can use `cat['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn'].df` to inspect the assets/files for this key.
                 

In [266]:
#del subset_dict['CMIP.IPSL.IPSL-CM5A2-INCA.historical.Omon.gn']
#del subset_dict['CMIP.IPSL.IPSL-CM6A-LR-INCA.historical.Omon.gn'] 

In [277]:
subset_dict

{'CMIP.IPSL.IPSL-CM6A-LR.historical.Omon.gn': <xarray.Dataset>
 Dimensions:         (y: 332, x: 362, nvertex: 4, time: 1980, axis_nbounds: 2,
                      member_id: 1, dcpp_init_year: 1)
 Coordinates:
     bounds_nav_lat  (y, x, nvertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
     bounds_nav_lon  (y, x, nvertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
     nav_lat         (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
     nav_lon         (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
   * time            (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T...
     time_bounds     (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(1980, 2), meta=np.ndarray>
   * member_id       (member_id) object 'r1i1p1f1'
   * dcpp_init_year  (dcpp_init_year) float64 nan
 Dimensions without coordinates: y, x, nvertex, axis_nbounds
 Data variables:
     area            (y, x) float32 dask.array<chunksi

In [279]:
## Create a new dictionary for manipulating

# initialize
cmip6_sst=dict()
models_to_del=[]
nlat_threshold=90

for key in subset_dict.keys():
    ## create shortened key name that only includes institution and source id
    _, inst_id, src_id, _, _, _ = key.split(sep='.')
    short_key=inst_id + '.' + src_id
    
    ## clean up dataset info
    ds=subset_dict[key].tos[0] # specifying 0 uses only the first member_id
    # make sure names of coordinate variables match
    ds=match_lat_lon_names(ds)
    # make a list of models with low resolution (<2.5°), for later
    if np.size(ds.lat)<nlat_threshold:
        models_to_del.append(short_key)
    # add a source_id coordinate
    ds=add_source_id_coord(subset_dict[key]) #ds.assign_coords(source_id=src_id)
    
    ## process data
    # Trim the time range
    #ds=ds.sel(time=slice('1948','2014'))
    # calculate monthly climatologies
    ds=monthly_climatology(ds)
    
    ## update new dictionary with processed data
    cmip6_sst[short_key]=ds

# delete low res models
for model in models_to_del:
    del cmip6_sst[model]

In [296]:
cmip6_sst

{'IPSL.IPSL-CM6A-LR': <xarray.Dataset>
 Dimensions:         (month: 12, y: 332, x: 362, nvertex: 4, member_id: 1,
                      dcpp_init_year: 1)
 Coordinates:
     bounds_nav_lat  (y, x, nvertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
     bounds_nav_lon  (y, x, nvertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
     nav_lat         (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
     nav_lon         (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
   * member_id       (member_id) object 'r1i1p1f1'
   * dcpp_init_year  (dcpp_init_year) float64 nan
     source_id       <U12 'IPSL-CM6A-LR'
   * month           (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
 Dimensions without coordinates: y, x, nvertex
 Data variables:
     area            (month, y, x) float32 dask.array<chunksize=(1, 332, 362), meta=np.ndarray>
     tos             (month, member_id, dcpp_init_year, y, x) float32 dask.array<chunksize=(1, 1, 1

In [299]:
## Create a new dictionary for storing regridded data

# initialize
cmip6_sst_same_grid=dict()

# Define standard coordinates based on lowest resolution model
lats = cmip6_sst['NASA-GISS.GISS-E2-1-G'].lat.values
lons = cmip6_sst['NASA-GISS.GISS-E2-1-G'].lon.values

# interp model grids to lower resolution
# all models are on a 0:360 grid so no need to convert lons
for key,ds in cmip6_sst.items():
    dask_array.compute()
    ds['nav_lon']=test_lon
    ds_lr=ds.interp(lat=lats, lon=lons)
    ds_lr_clean=drop_all_bounds(ds_lr)
    cmip6_sst_same_grid[key]=ds_lr_clean

NameError: name 'dask_array' is not defined

In [ ]:
## Create a new xarray dataset for storing regridded data

# initialize
cmip6_sst_same_grid=[]

for index, (key, ds) in enumerate(cmip6_sst.items()):
    # interp to standard grid
    ds_lr=ds.interp(lat=lats, lon=lons)
    ds_lr_clean=drop_all_bounds(ds_lr)
    # concatenate
    if index==0:
        cmip6_sst_same_grid=ds_lr_clean
    else:
        cmip6_sst_same_grid=xr.concat([cmip6_sst_same_grid,ds_lr_clean], dim='source_id', coords='minimal')

# clean
#cmip6_sst_same_grid=cmip6_sst_same_grid.drop(['bnds','member_id','dcpp_init_year'])
#cmip6_sst_same_grid=cmip6_sst_same_grid.drop_dims(['bnds','member_id','dcpp_init_year'])

In [ ]:
# save output
cmip6_sst_same_grid.to_netcdf('cmip6.sst.climo.nc')

### old, didn't work

In [226]:
def drop_all_bounds(ds):
    drop_vars = [vname for vname in ds.coords
                 if ( (('_bounds') in vname) or 
                     (('_bnds') in vname) )]
    return ds.drop(drop_vars)


def open_dset(df,i=0):
    # get the path to a specific zarr store
    zstore = df.zstore.values[i]
    # create a mutable-mapping-style interface to the zarr store
    mapper = fsspec.get_mapper(zstore)
    # open it using xarray and zarr
    ds = xr.open_zarr(mapper, consolidated=True)
    # make sure lat and lon var names match
    ds = match_lat_lon_names(ds)
    # add source id as a coord to concatenate along
    ds = add_source_id_coord(ds)
    # drop unncessary coordinates
    ds_clean = drop_all_bounds(ds)
    return ds_clean

In [ ]:
## Store results in to a dataset dictionary
kwargs = {'aggregate':False,
          'zarr_kwargs': {'consolidated':True, 'use_cftime':True},
          'preprocess':combined_preprocessing }

model_dict = subset.to_dataset_dict()
list(model_dict.keys())

In [ ]:
dsets = {}

# iterate over the key-value pairs
for key,ds in model_dict.items():
    ds = ds.load()
    ds=model_dict[key]
    ds = match_lat_lon_names(ds)
    ds=drop_all_bounds(ds)
    ds=ds.drop_dims('dcpp_init_year')
    ds=add_source_id_coord(ds)
    
    ds_time_mean=calc_climatology(ds,season='ann')
    
    dsets=xr.concat([dsets,ds_time_mean], dim='source_id')
    
    # subset_cat = xr.concat(list(model_dict.values()), dim='source_id', coords='minimal')

In [ ]:
pr2d = []

for n in np.arange(0,len(model_dict.keys())):
    ds = open_dset(subset.df,i=n)
    ds_avg = annual_climatology(ds)
    if n==0:
        pr2d = ds_avg
    else:
        pr2d=xr.concat([pr2d,ds_avg], dim='source_id', coords='minimal')
    #ds_avg.load()

In [ ]:
pr2d.mean(dim='source_id').pr.plot()